<a href="https://colab.research.google.com/github/Dwinurcahyo2/Data-Mining-Lanjut/blob/main/UAS_data_mining_lanjut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**UAS DATA MINING LANJUT**

LIBRARY

In [ ]:
# ===============================
#  1. Import Library
# ===============================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


LOAD DATA

In [ ]:
# ===============================
# Load Dataset
# ===============================

df = pd.read_csv("uas_dml_dataset.csv")
df.head()

EDA

In [ ]:
# ===============================
# Exploratory Data Analysis (EDA)
# ===============================

df.info()
df.describe()
df.isnull().sum()

# Distribusi target
sns.countplot(data=df, x='target')
plt.title("Distribusi Kelas Target")
plt.show()

# Korelasi
plt.figure(figsize=(10, 8))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm')
plt.title("Matriks Korelasi")
plt.show()


DATA CLEANING

In [ ]:
# ===============================
# Data Cleaning
# ===============================

# Mengisi nilai kosong 
imputer = SimpleImputer(strategy='mean')
df[df.select_dtypes(include=[np.number]).columns] = imputer.fit_transform(df.select_dtypes(include=[np.number]))


df[df.select_dtypes(include=['object']).columns] = df.select_dtypes(include=['object']).fillna(df.mode().iloc[0])
df.isnull().sum()


PREPROCESSING

In [ ]:
# ===============================
# Preprocessing
# ===============================

# Label Encoding untuk fitur kategorikal
label_encoders = {}
for col in df.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Fitur dan target
X = df.drop(columns='target')  
y = df['target']

# Scaling fitur
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


MODELING & TRAINING

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)

TESTING & EVALUATION

In [ ]:
# ===============================
# Testing & Evaluation
# ===============================

# Prediksi
y_pred = model.predict(X_test)

# Evaluasi
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nAkurasi:")
print(accuracy_score(y_test, y_pred))

# Confusion matrix visual
plt.figure(figsize=(6, 4))
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.xlabel("Prediksi")
plt.ylabel("Aktual")
plt.title("Confusion Matrix")
plt.show()
